In [7]:
# =============================================================
# 0.  SET-UP  (run exactly once after every kernel restart)
# =============================================================
%matplotlib widget         
import matplotlib, matplotlib.pyplot as plt
matplotlib.rcParams['animation.html']        = 'jshtml'
matplotlib.rcParams['animation.embed_limit'] = 100   # MB for big graphs

import networkx as nx, time, threading, ipywidgets as widgets
from matplotlib.animation import FuncAnimation, FFMpegWriter, PillowWriter
from IPython.display   import HTML, display, clear_output


In [8]:
# =============================================================
# 1.  GRAPH – HELPERS
# =============================================================

def build_n_pan(n: int) -> nx.Graph:
    """Return the n-pan graph (cycle C_n with one pendant)."""
    G = nx.cycle_graph(range(1, n + 1))
    G.add_node(0)
    G.add_edge(0, 1)
    return G

def compute_base_coloring(n: int) -> dict[int, int]:
    """χ₌(C⁺¹_n) = 2 (n even) or 3 (n odd); return one such coloring."""
    base = {}
    if n % 2 == 0:                           # even n  →  2 colours
        for i in range(1, n + 1):
            base[i] = 1 if i % 2 else 2
        base[0] = 2
    else:                                    # odd n  →  3 colours
        r = n % 3
        if r == 0:                                # n ≡ 0 (mod 3)
            for i in range(1, n + 1):
                base[i] = ((i - 1) % 3) + 1
            base[0] = 3
        elif r == 1:                              # n ≡ 1 (mod 3)
            for i in range(1, n):
                base[i] = ((i - 1) % 3) + 1
            base[n] = 2
            base[0]  = 3
        else:                                     # n ≡ 2 (mod 3)
            for i in range(1, n - 1):
                base[i] = ((i - 1) % 3) + 1
            base[n - 1] = 1
            base[n]     = 2
            base[0]     = 3
    return base

def equitable_anim(n: int, m: int) -> FuncAnimation:
    """Return 4-frame animation revealing colour classes one by one."""
    G_room  = build_n_pan(n)
    base    = compute_base_coloring(n)
    H_corr  = nx.path_graph(range(1, m + 1))
    GH      = nx.cartesian_product(G_room, H_corr)

    k = max(base.values())                   # 2 or 3
    colouring = {(u, v): ((base[u] + v - 1) % k or k) for u, v in GH.nodes()}
    groups    = {g: [node for node, c in colouring.items() if c == g]
                 for g in range(1, k + 1)}

    # fixed layout for repeatability (seed)
    pos = nx.spring_layout(GH, seed=42)
    cmap = {0: 'lightgray', 1: 'tab:red', 2: 'tab:green', 3: 'tab:blue'}
    titles = ["Step 0 : all lamps grey",
              "Step 1 : class 1", "Step 2 : class 2", "Step 3 : class 3"]

    fig, ax = plt.subplots(figsize=(6, 5))
    plt.close(fig)                           # prevent duplicate blank plot

    def draw(frame: int):
        ax.clear()
        vis = set().union(*(groups[g] for g in range(1, frame + 1)))
        colours = [cmap[colouring[v]] if v in vis else cmap[0] for v in GH.nodes]
        nx.draw(GH, pos, node_color=colours, with_labels=True, ax=ax)
        ax.set_title(f"n = {n}, m = {m}   —   {titles[frame]}", fontsize=12)

    return FuncAnimation(fig, draw, frames=4, interval=800, repeat=False)


In [9]:
# =============================================================
# 2.  PREVIEW PANEL  (sliders + "Run Animation")
# =============================================================
n_slider  = widgets.IntSlider(value=5, min=3, max=15, step=1, description='n')
m_slider  = widgets.IntSlider(value=4, min=2, max=12, step=1, description='m')
run_btn   = widgets.Button(description='Run Animation')
out_area  = widgets.Output()

def _run_preview(_):
    with out_area:
        clear_output(wait=True)
        ani = equitable_anim(int(n_slider.value), int(m_slider.value))
        display(HTML(ani.to_jshtml()))

run_btn.on_click(_run_preview)
display(widgets.VBox([n_slider, m_slider, run_btn, out_area]))


In [14]:
save_current_preview()


✔ MP4 saved → pan_path_n7_m4.mp4


In [15]:
# =============================================================
# 3.  GENERATOR DISPATCHER  (start/stop buttons)
# =============================================================
class Dispatcher:
    """Rotate generators in a background thread."""
    def __init__(self, groups: dict[int, list], seconds_per_slot: int = 60):
        self.groups  = groups
        self.period  = seconds_per_slot
        self._thr    = None
        self._stop   = threading.Event()

    def _loop(self):
        order = list(self.groups)            # 1,2,…,k
        while not self._stop.is_set():
            for g in order:
                lamps = self.groups[g]
                print(f"[{time.strftime('%H:%M:%S')}] "
                      f"Generator {g} ON   ({len(lamps)} lamps)")
                if self._stop.wait(self.period):
                    break

    def start(self):
        if self._thr and self._thr.is_alive():
            return                          # already running
        self._stop.clear()
        self._thr = threading.Thread(target=self._loop, daemon=True)
        self._thr.start()
        print("Dispatcher started.")

    def stop(self):
        self._stop.set()
        if self._thr: self._thr.join()
        print("Dispatcher stopped.")

# helper to build full floor + colouring
def build_coloured_graph(n: int, m: int):
    G  = nx.cartesian_product(build_n_pan(n), nx.path_graph(range(1, m + 1)))
    k  = max(compute_base_coloring(n).values())
    col = {(u, v): ((compute_base_coloring(n)[u] + v - 1) % k or k)
           for u, v in G.nodes}
    return G, col, k

# small UI
start_btn = widgets.Button(description='Start Dispatcher', button_style='success')
stop_btn  = widgets.Button(description='Stop Dispatcher',  button_style='danger')

def _start(_):
    n_val, m_val = int(n_slider.value), int(m_slider.value)
    _, cmap, k   = build_coloured_graph(n_val, m_val)
    groups = {g: [v for v, c in cmap.items() if c == g] for g in range(1, k + 1)}
    for g, lamps in groups.items():
        print(f"Generator {g}: {len(lamps)} lamps")
    global DISP                           # keep alive across callbacks
    DISP = Dispatcher(groups, seconds_per_slot=60)
    DISP.start()

def _stop(_):
    if 'DISP' in globals():
        DISP.stop()

start_btn.on_click(_start)
stop_btn.on_click(_stop)
display(widgets.HBox([start_btn, stop_btn]))


In [16]:
# =============================================================
# 4.  OPTIONAL: SAVE THE CURRENT PREVIEW (MP4 if possible, else GIF)
# =============================================================
from matplotlib.animation import FFMpegWriter, PillowWriter

def save_current_preview():
    """Export the slider-selected preview to MP4 (or GIF if ffmpeg is absent)."""
    n_val, m_val = int(n_slider.value), int(m_slider.value)
    ani     = equitable_anim(n_val, m_val)

    mp4     = f"pan_path_n{n_val}_m{m_val}.mp4"
    gif     = f"pan_path_n{n_val}_m{m_val}.gif"

    # try MP4 first
    try:
        writer = FFMpegWriter(fps=1, bitrate=1800)
        ani.save(mp4, writer=writer, dpi=150)
        print("✔ MP4 saved →", mp4)
    except Exception as e:
        print("⚠ MP4 failed, switching to GIF.  Reason:", e)
        ani.save(gif, writer=PillowWriter(fps=1), dpi=150)
        print("✔ GIF saved →", gif)
